# CrewAI Bedrock Agent Core Memory Demo

This notebook demonstrates the core save and search functionality of AWS Bedrock Agent Core memory integration with CrewAI.

## Features

- Saving information to Agent Core memory
- Searching and retrieving stored information
- Memory enhancing conversational context
- Real-world conversation scenarios with memory persistence

## Prerequisites

### 1. AWS Setup
- AWS credentials configured (`aws configure` or environment variables)
- AWS Bedrock Agent Core Memory resource created
- Required IAM permissions:
  - `bedrock-agentcore:CreateEvent`
  - `bedrock-agentcore:ListEvents` 
  - `bedrock-agentcore:RetrieveMemories`
  - `bedrock-agentcore-control:CreateMemory` -> If you don't have a memory resrouce pre-configured
  - `bedrock-agentcore-control:GetMemory` -> If you don't have a memory resrouce pre-configured



### 2. Dependencies
```bash
pip install crewai[agentcore] boto3 bedrock_agentcore
```

## 1. Initial Setup and Configuration

In [ ]:
# Import required libraries
from crewai.memory.external.external_memory import ExternalMemory
from crewai.memory.storage.bedrock_agentcore_storage import (
    BedrockAgentCoreConfig,
    BedrockAgentCoreStrategyConfig,
    BedrockAgentCoreStorage,
)
from crewai import Agent, Task, Crew, LLM, Process
from bedrock_agentcore.memory.client import MemoryClient

import boto3
import time

print("✅ Libraries imported successfully")

In [ ]:
# Configuration - Update these with your actual values
MEMORY_NAME = "SaveSearchDemo"
ACTOR_ID = "demo-user-456"  # Unique identifier for the user
SESSION_ID = "save-search-session"  # Unique session identifier
REGION = "us-west-2"  # AWS region

print("Configuration:")
print(f"  Memory Name: {MEMORY_NAME}")
print(f"  Actor ID: {ACTOR_ID}")
print(f"  Session ID: {SESSION_ID}")
print(f"  Region: {REGION}")

In [ ]:
# Verify AWS credentials
try:
    session = boto3.Session()
    credentials = session.get_credentials()
    if credentials is None:
        raise Exception("No AWS credentials found")

    # Test basic access
    sts = session.client("sts")
    identity = sts.get_caller_identity()
    print(f"✅ AWS credentials verified for account: {identity.get('Account')}")

except Exception as e:
    print(f"❌ AWS credentials issue: {e}")
    print("Please configure your AWS credentials before proceeding.")

## 2. Create Agent Core Memory Storage

We'll create a memory instance with conversation and preference strategies for enhanced functionality.

In [ ]:
# Create Memory using Memory Client
# Note: Skip if you already have a Bedrock Agent Core Memory resource created
memory_client = MemoryClient(region_name=REGION)

# Create memory with conversation and preference strategies
response = memory_client.create_memory_and_wait(
    name=MEMORY_NAME,
    strategies=[
        {
            "summaryMemoryStrategy": {
                "name": "ConversationSummarizer",
                "namespaces": ["/conversations/{actorId}/{sessionId}"],
            }
        },
        {
            "semanticMemoryStrategy": {
                "name": "UserPreferences",
                "namespaces": ["/preferences/{actorId}"],
            }
        },
    ],
)

print(
    f"Memory instance created with ARN{response['arn']} and {len(response['strategies'])} strategies"
)

In [ ]:
# Extract required variables from response
MEMORY_ID = response["id"]
CONVERSATION_STRATEGY_ID = response["strategies"][0]["strategyId"]
PREFERENCE_STRATEGY_ID = response["strategies"][1]["strategyId"]

# Note: Uncomment below if you already have a memory resource created:
# MEMORY_ID = "memory-id"                                 # Replace with actual ID
# CONVERSATION_STRATEGY_ID = "conversation-strategy-id"   # Replace with actual ID
# PREFERENCE_STRATEGY_ID = "preference-strategy-id"       # Replace with actual ID

print(f"Memory ID: {MEMORY_ID}")
print(f"Conversation Strategy ID: {CONVERSATION_STRATEGY_ID}")
print(f"Preference Strategy ID: {PREFERENCE_STRATEGY_ID}")

In [ ]:
# Create config for Bedrock AgentCore external memory
config = BedrockAgentCoreConfig(
    memory_id=MEMORY_ID,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
    region_name=REGION,
    strategies=[
        BedrockAgentCoreStrategyConfig(
            name="conversation_summaries",
            namespaces=[
                BedrockAgentCoreStorage.resolve_namespace_template(
                    "/conversations/{actorId}/{sessionId}",
                    ACTOR_ID,
                    SESSION_ID,
                    CONVERSATION_STRATEGY_ID,
                )
            ],
            strategy_id=CONVERSATION_STRATEGY_ID,
        ),
        BedrockAgentCoreStrategyConfig(
            name="user_preferences",
            namespaces=[
                BedrockAgentCoreStorage.resolve_namespace_template(
                    "/preferences/{actorId}",
                    ACTOR_ID,
                    SESSION_ID,
                    PREFERENCE_STRATEGY_ID,
                )
            ],
            strategy_id=PREFERENCE_STRATEGY_ID,
        ),
    ],
)

# Create external memory instance
memory = ExternalMemory(
    embedder_config={
        "provider": "agentcore",
        "config": config,  # Convert AgentCoreConfig to dict
    }
)

print("Agent Core memory storage created and initialized")
print("   Provider: agentcore")
print(f"   Memory ID: {MEMORY_ID}")
print(f"   Actor: {ACTOR_ID}")
print(f"   Session: {SESSION_ID}")
print(f"   Strategies: {len(config.strategies)} configured")
for strategy in config.strategies:
    print(f"     - {strategy.name}: {strategy.strategy_id}")
    print(f"       Resolved namespaces: {strategy.namespaces}")

In [ ]:
# Create a simple crew to initialize memory
llm = LLM(
    model="bedrock/anthropic.claude-3-5-sonnet-20241022-v2:0", region_name="us-west-2"
)

memory_agent = Agent(
    role="Memory Assistant",
    goal="Demonstrate memory save and search capabilities",
    backstory="""You are a helpful assistant that demonstrates how memory 
    works by saving and retrieving information from conversations.""",
    verbose=False,
    allow_delegation=False,
    llm=llm,
)

memory_task = Task(
    description="Process user input and demonstrate memory functionality.",
    agent=memory_agent,
    expected_output="Acknowledgment of memory operation.",
)

# Initialize crew with memory
crew = Crew(
    agents=[memory_agent],
    tasks=[memory_task],
    process=Process.sequential,
    verbose=False,
    external_memory=memory,
)

# Set crew for memory management
memory.set_crew(crew)

print("✅ Memory assistant crew created and initialized")

## 3. Demonstrate Save Functionality

Let's save different types of information that would be typical in a conversational AI scenario.

In [ ]:
print("💾 Saving conversation information to memory...")
print("=" * 50)

# Simulate a realistic conversation with user preferences and context
conversation_data = [
    {
        "content": "Hi, I'm Sarah Chen, a software engineer working on machine learning projects. I prefer detailed technical explanations.",
        "metadata": {
            "type": "introduction",
            "category": "user_info",
            "role": "user",
            "timestamp": time.time(),
            "message_id": "msg-1",
            "session": SESSION_ID,
        },
    },
    {
        "content": "I'm currently working on a recommendation system using collaborative filtering and need help with data preprocessing.",
        "metadata": {
            "type": "project_context",
            "category": "technical",
            "role": "user",
            "timestamp": time.time(),
            "message_id": "msg-2",
            "session": SESSION_ID,
        },
    },
    {
        "content": "I prefer Python over R for data analysis, and I usually work with pandas and scikit-learn libraries.",
        "metadata": {
            "type": "preferences",
            "category": "technical_preferences",
            "role": "user",
            "timestamp": time.time(),
            "message_id": "msg-3",
            "session": SESSION_ID,
        },
    },
    {
        "content": "My dataset has about 1 million user-item interactions with sparse ratings. I'm concerned about memory efficiency.",
        "metadata": {
            "type": "technical_details",
            "category": "project_specifics",
            "role": "user",
            "timestamp": time.time(),
            "message_id": "msg-4",
            "session": SESSION_ID,
        },
    },
    {
        "content": "I work remotely from Seattle and prefer morning meetings. My timezone is PST.",
        "metadata": {
            "type": "personal_info",
            "category": "logistics",
            "role": "user",
            "timestamp": time.time(),
            "message_id": "msg-5",
            "session": SESSION_ID,
        },
    },
]

# Save all conversation data in a single batch operation
print(f"   Saving {len(conversation_data)} messages ...")
memory.save(conversation_data)

print("\n✅ All conversation data saved successfully!")
print(
    "   Information includes: user intro, project context, preferences, technical details, and personal info"
)

## 4. Demonstrate Search Functionality

Now let's search for the information we just saved using different types of queries.

In [8]:
def search_and_display(query: str, limit: int = 3, description: str = ""):
    """Search memory and display results in a formatted way"""
    print(f"🔍 {description}")
    print(f"Query: '{query}'")
    print("-" * 60)

    try:
        results = memory.search(query=query, limit=limit)

        if not results:
            print("   No results found")
            return []

        print(f"   Found {len(results)} result(s):\n")

        for i, result in enumerate(results, 1):
            content = result.get("context", "")[:120]
            score = result.get("score", "N/A")

            print(f"   Result {i} (Score: {score:.3f}):")
            print(f"     {content}...")

            # Display metadata if available
            metadata = result.get("metadata", {})
            if metadata:
                relevant_meta = {
                    k: v
                    for k, v in metadata.items()
                    if k in ["type", "category", "role"]
                }
                if relevant_meta:
                    print(f"     Metadata: {relevant_meta}")
            print()

        return results

    except Exception as e:
        print(f"   ❌ Search error: {str(e)}")
        return []

In [ ]:
# Test 1: Search for user information
search_and_display(
    "Sarah Chen software engineer",
    limit=2,
    description="Searching for user identity and background",
)
print("\n" + "=" * 70 + "\n")

In [ ]:
# Test 2: Search for technical preferences
search_and_display(
    "Python pandas scikit-learn preferences",
    limit=2,
    description="Searching for technical preferences and tools",
)
print("\n" + "=" * 70 + "\n")

In [ ]:
# Test 3: Search for project context
search_and_display(
    "recommendation system collaborative filtering",
    limit=2,
    description="Searching for current project information",
)
print("\n" + "=" * 70 + "\n")

In [ ]:
# Test 4: Search for technical challenges
search_and_display(
    "million interactions sparse ratings memory efficiency",
    limit=2,
    description="Searching for technical challenges and constraints",
)
print("\n" + "=" * 70 + "\n")

In [ ]:
# Test 5: Search for personal/logistics information
search_and_display(
    "Seattle timezone PST morning meetings",
    limit=2,
    description="Searching for personal and logistics preferences",
)
print("\n" + "=" * 70 + "\n")

## 5. Simulate Realistic User Queries

Now let's simulate realistic user queries that would benefit from memory context, similar to the conversation messages in the interactive test.

In [ ]:
# Create a more sophisticated agent for handling user queries
conversation_agent = Agent(
    role="AI Assistant",
    goal="Provide helpful responses using memory context from previous conversations",
    backstory="""You are an intelligent AI assistant with access to conversation 
    history and user preferences. Use this memory to provide personalized, 
    contextual responses that reference previous interactions.""",
    llm=llm,
    verbose=True,
)

conversation_task = Task(
    description="""Answer the user's question: {question}
    
    Use memory to recall relevant information about the user, their preferences, 
    current projects, and previous conversations to provide a personalized response.""",
    expected_output="A helpful, personalized response that demonstrates memory usage.",
    agent=conversation_agent,
)

conversation_crew = Crew(
    agents=[conversation_agent],
    tasks=[conversation_task],
    external_memory=memory,
    verbose=True,
)

print("✅ Conversation crew created with memory integration")

In [31]:
def ask_question(question: str):
    """Ask a question and get a memory-enhanced response"""
    print(f"❓ User Question: {question}")
    print("=" * 60)

    result = conversation_crew.kickoff({"question": question})

    print("\n🤖 AI Response:")
    print(result.raw)
    print("\n" + "=" * 80 + "\n")

    return result

In [ ]:
# Query 1: Ask about user preferences (similar to interactive_test.py)
ask_question(
    "What do you know about my technical preferences and the tools I like to use?"
)


# Sample Crew Output
# "Sarah, from our previous interactions, I recall that you are working on a recommendation system using
# collaborative filtering and are particularly focused on data preprocessing. You prefer Python for your data analysis tasks
# and typically use libraries such as pandas and scikit-learn. Given the size of your dataset, which contains approximately
# 1 million user-item interactions with sparse ratings, you're also concerned about memory efficiency.
# If you need further assistance with optimizing memory usage or any other aspect of your project, feel free to ask!"

In [ ]:
# Query 2: Ask about current project
ask_question(
    "Can you remind me what I'm currently working on and what challenges I mentioned?"
)

# Sample Response:
# Hi Sarah, you're currently working on developing a recommendation system using collaborative filtering techniques.
# Your main focus is on handling a large dataset of 1 million user-item interactions with sparse ratings, and
# you're particularly concerned about memory efficiency. You mentioned challenges with data preprocessing and
# dealing with the sparsity of the dataset. Given your preference for Python, you're likely using pandas and scikit-learn
# libraries for your tasks. If you need further assistance or specific strategies for optimizing memory usage or improving
# your preprocessing steps, feel free to ask!

In [ ]:
# Query 3: Ask about personal information
ask_question("What do you remember about my background and where I work from?")

# Sample Response:
# Sarah, I recall that you are a software engineer working on machine learning projects, with a preference
# for detailed technical explanations. You are currently developing a recommendation system using collaborative
# filtering and are dealing with a dataset of approximately 1 million user-item interactions that has sparse ratings.
# You're particularly focused on memory efficiency in handling this large dataset. You prefer Python, especially using
# pandas and scikit-learn libraries. Additionally, you work remotely from Seattle in the PST timezone and prefer scheduling
# meetings in the morning. If you need further assistance with data preprocessing or any other aspect of your project,
# feel free to ask!

In [ ]:
# Query 4: Ask for project-specific help (contextual)
ask_question(
    "Given my dataset size and memory concerns, what preprocessing approach would you recommend?"
)

In [ ]:
# Query 5: Ask about scheduling (using personal preferences)
ask_question(
    "When would be the best time to schedule a technical review meeting with me?"
)

## 7. Memory Analytics and Insights

Let's examine what's stored in memory and get some insights.

In [ ]:
# List recent memory events

memory_client = MemoryClient(region_name=REGION)


def list_memory_events(max_results: int = 10):
    """List recent events stored in Agent Core memory"""
    try:
        events = memory_client.list_events(
            memory_id=MEMORY_ID,
            session_id=SESSION_ID,
            actor_id=ACTOR_ID,
            max_results=max_results,
        )

        print(f"📋 Found {len(events)} events in Agent Core memory:")
        print("=" * 50)

        for i, event in enumerate(events, 1):
            print(f"\nEvent {i}: {event['eventId']}")
            print(f"  Timestamp: {event.get('eventTimestamp')}")

            for payload in event.get("payload", []):
                if "conversational" in payload:
                    conv = payload["conversational"]
                    role = conv.get("role", "UNKNOWN")
                    content = conv.get("content", {}).get("text", "")[:100]
                    print(f"  {role}: {content}...")

    except Exception as e:
        print(f"❌ Error listing events: {str(e)}")


list_memory_events()